In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'metrics')
sys.path.append(source_path)

from dl import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split,
    cont_cat_split
)
from metrics import calc_spread_return_sharpe

Global seed set to 42
Global seed set to 42


##  Get Data and train a Neural Network

In [ ]:
ROOT_PATH = 'c:/Users/gilbe/Documents/TokyoData'


'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
train_df['Date'] = pd.to_datetime(train_df['Date']) 
train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)
print(train_df.head())

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)

import matplotlib.pyplot as plt
print('Raw Time Series data shape:', train_df.shape)
print('No Unique Securities code:', train_df['SecuritiesCode'].nunique())

df_1301 = train_df[train_df['SecuritiesCode'] == 1301].drop(['SecuritiesCode', 'Volume'], axis=1)

df_1301 = date_features(df_1301)

cont, cat = cont_cat_split(df_1301, 'int64')
df_train_cat, df_val_cat = ts_split(cat)
df_train, df_val = ts_split(cont)

xtrain, ytrain = preprocess(df_train, 'Target', 1, continous_cols=['Close'])
xval, yval = preprocess(df_val, 'Target', 1, continous_cols=['Close'])

print('xtrain.shape:', xtrain.shape)
print(xtrain[:5])
print()
print('ytrain.shape:', ytrain.shape)
print(ytrain[:5])
print('df_train_cat.shape:', df_train_cat.shape)
print(df_train_cat.head())

""" xtrain and df_train_cat have different shapes!!!!!"""

                    RowId  SecuritiesCode    Open    High     Low   Close  \
Date                                                                        
2017-01-04  20170104_1301            1301  2734.0  2755.0  2730.0  2742.0   
2017-01-04  20170104_1332            1332   568.0   576.0   563.0   571.0   
2017-01-04  20170104_1333            1333  3150.0  3210.0  3140.0  3210.0   
2017-01-04  20170104_1376            1376  1510.0  1550.0  1510.0  1550.0   
2017-01-04  20170104_1377            1377  3270.0  3350.0  3270.0  3330.0   

             Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  \
Date                                                                       
2017-01-04    31400               1.0               NaN            False   
2017-01-04  2798500               1.0               NaN            False   
2017-01-04   270800               1.0               NaN            False   
2017-01-04    11300               1.0               NaN            False   
2017

### Train the model

In [ ]:

%%time
import torch
from sklearn.impute import SimpleImputer


imp = SimpleImputer(missing_values=np.nan, strategy='mean')
batch_size = 64
train_dataloader = get_loader(x=xtrain, y=ytrain, batch_size=batch_size, x_cat=df_train_cat.to_numpy())
val_dataloader = get_loader(x=xval, y=yval, batch_size=batch_size, x_cat=df_val_cat.to_numpy())


cat_features = 3 
embedding_dim = 10
# cat_features = cat_features * embedding_dim
# print('in_features:', xtrain.shape[1] + cat_features)

model = NeuralNetwork(
    in_features=xtrain.shape[1], 
    units=1024,
    out_features=1, 
    categorical_dim=cat_features,
    no_embedding=len(df_train_cat), 
    emb_dim=embedding_dim
)

print(model)

trainer = Trainer(model, lr=3.3e-6)
trainer.fit_epochs(train_dataloader, val_dataloader, use_cyclic_lr=True, x_cat=True, epochs=25)
